In [2]:
import pandas as pd
d = {"headers": {"Stadium": ["id", "visit_date", "people"]}, "rows": {"Stadium": [[1, "2017-01-01", 10], [2, "2017-01-02", 109], [3, "2017-01-03", 150], [4, "2017-01-04", 99], [5, "2017-01-05", 145], [6, "2017-01-06", 1455], [7, "2017-01-07", 199], [8, "2017-01-09", 188]]}}
pd.DataFrame(d['rows']['Stadium'], columns=d['headers']['Stadium']).to_csv("./Stadium.txt", index=None)

In [3]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("human traffic of stadium").config("pyspark.sql.shuffle.partition","4").getOrCreate()

In [5]:
s = spark.read.option("header", True).csv("./Stadium.txt")
s = s.withColumn("id", s.id.cast('int')).withColumn('visit_date', s.visit_date.cast('date')).withColumn('people', s.people.cast('int'))
s.show()

+---+----------+------+
| id|visit_date|people|
+---+----------+------+
|  1|2017-01-01|    10|
|  2|2017-01-02|   109|
|  3|2017-01-03|   150|
|  4|2017-01-04|    99|
|  5|2017-01-05|   145|
|  6|2017-01-06|  1455|
|  7|2017-01-07|   199|
|  8|2017-01-09|   188|
+---+----------+------+



Write an SQL query to display the records with three or more rows with consecutive id's, and the number of people is greater than or equal to 100 for each.

Return the result table ordered by visit_date in ascending order.

The query result format is in the following example.

In [15]:
from pyspark.sql.window import Window
from pyspark.sql.functions import row_number, col
Window_spec = Window.orderBy('id')
tmp = s.where('people>=100').select("*", (s.id-row_number().over(Window_spec)).alias("row"))
tmp.groupBy('row').count().where('count>=3').join(tmp, 'row', 'inner').select('id','visit_date','people').orderBy('visit_date', ascending=True).show()

22/07/10 11:42:06 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
22/07/10 11:42:06 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.


+---+----------+------+
| id|visit_date|people|
+---+----------+------+
|  5|2017-01-05|   145|
|  6|2017-01-06|  1455|
|  7|2017-01-07|   199|
|  8|2017-01-09|   188|
+---+----------+------+

